In [1]:
'''
imports
'''

import pandas as pd
from scipy.fftpack import fft
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats 
import os
import cmath
import os
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler


In [10]:
#### making a 300*90 matrix !!!

def replace_ij(df):
    for tits in df.columns:
        if(tits != 'File'):
        
            if df[tits].dtypes =='O':
                df[tits] = df[tits].str.replace('i','j')
                df[tits] = df[tits].str.replace(' ','')
                df[tits] = df[tits].apply(lambda x: np.complex(x))
                
    return df



'''
This downsamples the csv file content with dsr: DOWN SAMPLING RATIO.
'''
def create_downsample(dsr,samples,csv):
    
    df = pd.read_csv(csv,index_col = 0 )
    print(df.head())
    df = replace_ij(df)
    length =  df.shape[1] ## 30 subcarriers
    #print(df.head)
    
    total = []
    
    temp = np.zeros((30000,length))
    temp = np.array(df[:samples])
    
    trans = np.transpose(temp)
    
    down_sampled = np.zeros((length,int(samples/dsr)),dtype = float)
      
    for row in range(0,dsr):
        down_sampled = trans[:,row::dsr]
        total.append(down_sampled)
    #print("The shape of desampled list :",np.shape(total))
    #total = np.asarray(total)
    return total
   
    

##########################################################
'''
 below code is for returning the list of col_names [b_1.....b_30]
'''

'''
This converts the (16,30,1875) to (172,30,300), default = 16
WITH AN OVERLAPPING WINDOW OF 50%
'''

def part_time_slice(arr,size = 300,overlap_per = 50):

    start = int(0)
    stop = int(start+size)
    temp = arr
    list1 = []
    while(stop <= arr.shape[1]):## arr.shape[1] == 1875. 
        datapoint = np.zeros((30,size))
        datapoint = np.array(temp[:,start:stop])
        #print("This is datapoint")
        #print(np.shape(datapoint))
        list1.append(datapoint)
        start =int( start+((100-overlap_per)*0.01*size))
        stop = int(start  +size)
    return list1 
'''
Joins every small part to the whole
'''

def total_time_parse(sliced_list,dsr = 16):
    time_parse_list = []
    for itr in range(dsr):
        temp = (sliced_list[itr])
        #print("The shape of temp is ", np.shape(temp))
        list_son = part_time_slice(temp)
        
        #print("The dimension of list_son is :", np.shape(list_son))
        
        time_parse_list.extend(list_son)
        
    return time_parse_list


In [3]:
### testing !!
one = '/home/surya/Desktop/wifi_sensing/final/2_BHK_rawdata/noact/13A_noactivity_2.csv'

dsr= 16 

samples = 30000

sliced_list = (create_downsample(dsr,samples,one))
time_parse = total_time_parse(sliced_list)
#print(time_parse[0])

      a_1      b_1      c_1      a_2      b_2      c_2      a_3      b_3  \
0   23+5i   -1-18i   -20-4i   25-19i  -19-15i  -21+19i    0-31i  -25+14i   
1  15+18i    8-17i  -17-14i    32-3i  -13-20i   -28+5i   16-30i   -23-4i   
2  -7-23i  -13+12i   11+19i  -29-11i   -2+25i    27+6i  -25+21i   22+15i   
3  -1+22i    16-6i   -2-18i   20+20i   10-21i  -23-18i    33-5i  -11-21i   
4   24-2i   -6-15i   -20+4i   19-23i   -24-8i  -14+28i   -9-28i  -16+17i   

       c_3      a_4  ...     c_27    a_28     b_28    c_28    a_29    b_29  \
0    5+31i  -27-17i  ...  -14+23i  11-10i  -24+32i  10+27i   2-11i   8+33i   
1  -10+28i  -13-29i  ...   -9+28i   7-11i   -8+36i  20+20i  -5-10i  21+26i   
2   21-22i   -4+34i  ...   26+17i  -5-11i   25+28i   25-6i   -9-3i   33-3i   
3   -30+7i   20-25i  ...   26-11i  -10+1i    38-6i  10-24i   -8+4i  18-28i   
4   16+27i  -32-10i  ...   13+26i   -1-9i   17+35i   29+4i   -9-4i   33+9i   

      c_29    a_30    b_30     c_30  
0    22+7i  -2-10i  17+20i    18-3i 

In [6]:
tempo = time_parse[0]
print(tempo.shape)

(90, 300)


In [4]:
# returns phase of a complex number
def phase(x):
    x = cmath.phase(x)
    return x
#covariance of the array
def covariance(array):
    array = (array - array.mean(axis = 0))/array.std(axis = 0)
    cov =np.cov(array.T)
    return cov

def eigen_decomposition(array):
    eig_values, eig_vectors = np.linalg.eig(array)
    eig_values = sorted(eig_values,reverse = True)
    return eig_values

def extract_features(array):
    a = array[0]
    b = array[1]
    return a,b

cols = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30']
def heat_map(array):
    plt.figure(figsize=(20,20))
    sns.set(font_scale=1.5)
    hm = sns.heatmap(array,cbar=True,annot=True,square=True,fmt='.2f',annot_kws={'size': 12},yticklabels=cols,xticklabels=cols)
    plt.tight_layout()
    plt.show()

#Returns n features
def get_features(array,n):
    phase_cov1 = covariance(array)
    phase_eig1 = eigen_decomposition(phase_cov1)
    features = phase_eig1[0:n]
    return features


#gets the phase difference arrays from 300*90
#The arrays returned should be passed into get_features to get features
def get_pd(array):
    array = array.applymap(phase)
    df1_a = array[['a_1','a_2','a_3','a_4','a_5','a_6','a_7','a_8','a_9','a_10','a_11','a_12','a_13','a_14','a_15','a_16','a_17','a_18','a_19','a_20','a_21','a_22','a_23','a_24','a_25','a_26','a_27','a_28','a_29','a_30']]
    df1_b = array[['b_1','b_2','b_3','b_4','b_5','b_6','b_7','b_8','b_9','b_10','b_11','b_12','b_13','b_14','b_15','b_16','b_17','b_18','b_19','b_20','b_21','b_22','b_23','b_24','b_25','b_26','b_27','b_28','b_29','b_30']]
    df1_c = array[['c_1','c_2','c_3','c_4','c_5','c_6','c_7','c_8','c_9','c_10','c_11','c_12','c_13','c_14','c_15','c_16','c_17','c_18','c_19','c_20','c_21','c_22','c_23','c_24','c_25','c_26','c_27','c_28','c_29','c_30']]
    df1_a.columns = [['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30']]
    df1_b.columns = [['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30']]
    df1_c.columns = [['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30']]
    ba_sub = df1_b.subtract(df1_a)
    cb_sub = df1_c.subtract(df1_b)
    ac_sub = df1_a.subtract(df1_c)
    return ba_sub,cb_sub,ac_sub

def fina_def(array,n):
    ba,cb,ac = get_pd(array)
    ba_eig = get_features(ba,n)
    cb_eig = get_features(cb,n)
    ac_eig = get_features(ac,n)
    eig = []
    eig.extend(ba_eig)
    eig.extend(cb_eig)
    eig.extend(ac_eig)
    return eig

In [7]:
df = pd.DataFrame(tempo.T)
if (len(df.columns) == 90):
    df.columns = ['a_1','b_1','c_1','a_2','b_2','c_2','a_3','b_3','c_3','a_4','b_4','c_4','a_5','b_5','c_5','a_6','b_6','c_6','a_7','b_7','c_7','a_8','b_8','c_8','a_9','b_9','c_9','a_10','b_10','c_10','a_11','b_11','c_11','a_12','b_12','c_12','a_13','b_13','c_13','a_14','b_14','c_14','a_15','b_15','c_15','a_16','b_16','c_16','a_17','b_17','c_17','a_18','b_18','c_18','a_19','b_19','c_19','a_20','b_20','c_20','a_21','b_21','c_21','a_22','b_22','c_22','a_23','b_23','c_23','a_24','b_24','c_24','a_25','b_25','c_25','a_26','b_26','c_26','a_27','b_27','c_27','a_28','b_28','c_28','a_29','b_29','c_29','a_30','b_30','c_30']
            
            
eig = fina_def(df,2)
print(eig)


def feature_produce(chunk):
    df = pd.DataFrame(chunk.T)
    if (len(df.columns) == 90):
        df.columns = ['a_1','b_1','c_1','a_2','b_2','c_2','a_3','b_3','c_3','a_4','b_4','c_4','a_5','b_5','c_5','a_6','b_6','c_6','a_7','b_7','c_7','a_8','b_8','c_8','a_9','b_9','c_9','a_10','b_10','c_10','a_11','b_11','c_11','a_12','b_12','c_12','a_13','b_13','c_13','a_14','b_14','c_14','a_15','b_15','c_15','a_16','b_16','c_16','a_17','b_17','c_17','a_18','b_18','c_18','a_19','b_19','c_19','a_20','b_20','c_20','a_21','b_21','c_21','a_22','b_22','c_22','a_23','b_23','c_23','a_24','b_24','c_24','a_25','b_25','c_25','a_26','b_26','c_26','a_27','b_27','c_27','a_28','b_28','c_28','a_29','b_29','c_29','a_30','b_30','c_30']
    eig = fina_def(df,2) 
    return eig
    

[9.131930078276614, 8.302145315443129, 5.832239567906465, 5.382572751630524, 10.615433322165433, 9.518771684786419]


In [141]:
def create_col(target):
    if(target == 0):
        directory = '/home/surya/Desktop/wifi_sensing/final/2_BHK_rawdata/noact'

        z = directory + "/"
    else:
        directory = '/home/surya/Desktop/wifi_sensing/final/2_BHK_rawdata/act'
        z = directory + "/"
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            a = z + filename
            
            df = pd.read_csv(a)
            if (len(df.columns) == 90):
                df.columns = ['a_1','b_1','c_1','a_2','b_2','c_2','a_3','b_3','c_3','a_4','b_4','c_4','a_5','b_5','c_5','a_6','b_6','c_6','a_7','b_7','c_7','a_8','b_8','c_8','a_9','b_9','c_9','a_10','b_10','c_10','a_11','b_11','c_11','a_12','b_12','c_12','a_13','b_13','c_13','a_14','b_14','c_14','a_15','b_15','c_15','a_16','b_16','c_16','a_17','b_17','c_17','a_18','b_18','c_18','a_19','b_19','c_19','a_20','b_20','c_20','a_21','b_21','c_21','a_22','b_22','c_22','a_23','b_23','c_23','a_24','b_24','c_24','a_25','b_25','c_25','a_26','b_26','c_26','a_27','b_27','c_27','a_28','b_28','c_28','a_29','b_29','c_29','a_30','b_30','c_30']
            df.to_csv(a,index = False)

create_col(0)
create_col(1)

In [8]:


def create_dad(csv,target,title,dsr= 16 ,samples = 30000):
    dad = pd.DataFrame()
    sliced_list = create_downsample(dsr,samples,csv)
    time_parse_list = total_time_parse(sliced_list) ## This time parse list is for each 
    print("The shape after time parsing : ",np.shape(time_parse_list))
    
    son = np.array(time_parse_list)
    filename = title
    content_csv = []
    for itr in range(son.shape[0]):
        row = []
        title = filename + "(" + str(itr) + ")"
        feat = np.round(feature_produce(son[itr]),decimals = 3)
        row.append(title)
        row.extend(feat)
        row.append(target)
        content_csv.append(row)
    dad =dad.append(content_csv)
    #print(dad.head)
        
        
    return dad


def create_godfather(dataframe,target):
    if(target == 0):
        GF = dataframe
        directory = '/home/surya/Desktop/LSTM/no_act'

        z = directory + "/"
    else:
        GF = dataframe
        directory = '/home/surya/Desktop/LSTM/act'
        z = directory + "/"
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            a = z + filename
# <><><><><><><><><><><><><><><><><><><><><><><><><><>

#This is for deciding the title


        if(target == 0):
            sstring1 = '/home/surya/Desktop/LSTM/no_act'
            res = a.replace(sstring1, 'no_act') 
        else:
            sstring1 = '/home/surya/Desktop/LSTM/act'
            res = a.replace(sstring1, '') 
                
        sstring2 = '.csv'
        title = res.replace(sstring2, '') 
            
        print('Under Progress',title)
        content_one_csv = create_dad(a,target,title)
        GF = GF.append(content_one_csv)
        
#<><><><><><><><><><><><><><><><><><><><><><><><><><>
    return GF


In [11]:
### Creation of feature sett

GF = pd.DataFrame()
GF = create_godfather(GF,0)
GF = create_godfather(GF,1)
print(GF.head())
print(len(GF.columns))
if (len(GF.columns) == 8):
    GF.columns = ['title','BA_1','BA_2','CB_1','CB_2','AC_1','AC_2','Y']
GF.to_csv("Phase_single_room.csv",index=False)
print('CSV_created!!!')

Under Progress no_act/csi_noactivity_1
       a_1     b_1    c_1      a_2      b_2    c_2     a_3     b_3    c_3  \
0    29-1i   1-14i  -1+4i   24-30i  -13-13i   3+5i  -9-38i  -20+2i   6+1i   
1  -19+16i   8+10i  -1-3i    1+32i    17+1i  -4-2i  28+19i  11-14i  -4+3i   
2    7+24i   13-2i     -3   30+13i    8-16i  -3+3i  29-18i  -9-16i   0+5i   
3   -24-8i  -5+12i   2-3i  -28+19i    9+16i   0-5i  -1+35i   20+2i  -5-2i   
4   -24-7i  -4+12i   2-2i  -28+18i    8+15i  -1-5i  -2+33i   18+1i  -5-2i   

       a_4  ...   c_27    a_28     b_28   c_28     a_29     b_29   c_29  \
0  -35-19i  ...  -5+8i  23-24i  -25-19i   4+9i   -2-27i   -26+5i   7+3i   
1    33-8i  ...  -3-8i  11+31i   24-14i  -8-2i    26+9i    4-23i  -5+4i   
2    6-33i  ...  -8+1i   32-3i   -7-26i  -4+7i   16-22i  -20-10i   3+7i   
3   27+23i  ...  -1-9i   1+34i    28-5i  -7-5i   22+18i   12-21i  -7+3i   
4   22+24i  ...   7-3i  -32+9i   11+25i   3-8i  -11+24i    22+6i  -3-6i   

      a_30     b_30   c_30  
0  -12-19i  -17+14

The shape after time parsing :  (176, 90, 300)
Under Progress /csi_walk_b2_1
     a_1      b_1    c_1    a_2      b_2    c_2    a_3      b_3    c_3    a_4  \
0   2-5i    2+24i   2-2i  -3-7i   26+18i  -1-3i  -8-2i   31-11i  -3-1i  -8+5i   
1   2-5i    2+24i   1-2i  -3-7i   26+19i  -1-3i  -7-3i    32-9i  -3-1i  -8+4i   
2   5-2i  -17+17i      2   3-6i    3+31i   2-2i  -4-7i   29+17i  -1-3i  -8-3i   
3  -1+5i   -2-23i  -1+1i   3+6i  -25-17i   1+3i   8+2i  -31+12i   3+1i   7-5i   
4   2-5i    1+25i   2-2i  -3-6i   26+19i   0-3i  -8-2i   33-12i  -3-2i  -7+6i   

   ...   c_27    a_28     b_28   c_28    a_29     b_29   c_29   a_30     b_30  \
0  ...   2-3i  -13+1i    33+1i  -2-3i   -7+8i   18-20i     -3  -2+9i    6-22i   
1  ...   3-2i  -11-8i   25+21i   1-3i  -11+3i    26-4i  -2-2i  -7+5i   19-12i   
2  ...      3  -7-10i   17+28i   1-3i  -10-1i    26+5i  -1-2i  -8+4i    22-6i   
3  ...   1+3i    9-9i  -24+19i   3+1i   0-10i   -1+26i   2-1i  -5-8i   10+20i   
4  ...  -2-3i  -5+11i   16-27i 

Under Progress /csi_walk_a6e2_1
      a_1     b_1    c_1      a_2     b_2    c_2     a_3      b_3   c_3  \
0   9+26i   16-1i  -3-1i   35+15i  12-18i  -4+3i  33-20i   -8-22i  0+6i   
1  14+22i   12-5i  -3+1i    34+2i   4-17i  -2+4i  20-29i  -13-13i  3+3i   
2  -25-4i  -4+12i   1-3i  -25+23i   9+15i  -2-4i   5+34i       18    -4   
3  22+13i    9-9i  -2+2i   32-12i  -3-17i   0+4i   9-33i   -16-7i  3+2i   
4  13-22i   -9-8i   2+3i  -14-32i  -17+4i   4-1i  -35-5i   -6+18i  2-4i   

       a_4  ...   c_27     a_28    b_28   c_28     a_29     b_29   c_29  \
0    8-37i  ...  -9-4i   29+20i  14-32i  -8+4i    28-9i  -14-26i  -1+8i   
1  -10-34i  ...  -2+8i    3-28i  -28+2i   6+6i  -17-16i  -13+20i   7-1i   
2   29+20i  ...   6-6i  -20+21i  24+16i  -1-9i    2+23i    22-7i  -6-4i   
3  -19-28i  ...  -4+7i    8-27i  -28-3i   4+7i  -12-19i  -17+16i   7+1i   
4  -26+23i  ...      9   -28+4i   9+29i   5-7i  -11+21i    23+8i  -2-7i   

     a_30     b_30   c_30  
0  18-18i  -20-15i   2+6i  
1  -19-5i 

       a_1     b_1    c_1     a_2      b_2    c_2      a_3     b_3    c_3  \
0    26+4i   3-13i  -2+2i  25-26i  -14-14i   1+3i   -7-35i  -19+4i   4+1i   
1  -15+22i   10+7i  -2-3i  10+34i    16-4i  -4-1i   35+11i   6-16i  -3+3i   
2    25-2i  -1-11i  -1+3i  18-29i   -13-9i   2+4i  -16-31i  -15+7i      5   
3  -19+22i   9+10i  -1-4i   6+41i    19-3i  -4-2i   35+19i   9-17i  -4+3i   
4   19-23i  -10-9i   2+4i  -7-37i   -17+3i   5+1i  -38-15i  -8+18i   4-4i   

       a_4  ...   c_27     a_28     b_28   c_28     a_29    b_29   c_29  \
0  -31-18i  ...   6+8i  -25-25i  -16+25i   9-1i   -29+4i   8+24i   4-7i   
1   31-18i  ...  -5-6i   18+24i   19-21i  -8+1i    26+1i  -3-24i  -5+5i   
2   -33-7i  ...   5+6i  -22-19i  -14+23i   8-2i   -23+5i   9+22i   3-6i   
3   36-15i  ...  -1-9i    3+34i    30-9i  -7-5i   22+16i  11-23i  -7+2i   
4  -35+20i  ...   9+3i   -31-8i   -2+30i   7-6i  -21+18i  20+17i   0-8i   

      a_30     b_30   c_30  
0  -20+14i   17+14i   0-6i  
1    20-9i  -11-17i  -2+5i  

The shape after time parsing :  (176, 90, 300)
Under Progress /csi_walk_a5b6_2
       a_1     b_1    c_1      a_2      b_2    c_2      a_3      b_3    c_3  \
0  -12+24i   13+6i  -2-3i   16+32i    17-9i     -4    36+5i    2-20i  -2+4i   
1  -26-11i  -5+14i   3-2i  -33+18i   10+17i   1-5i   -4+40i    21+2i  -5-3i   
2  -17+22i   11+9i  -2-3i    9+37i    19-5i  -5-2i   34+15i    7-19i  -4+4i   
3    5-27i  -14-2i   2+2i  -23-25i  -12+13i   4-1i   -38+6i    4+19i   2-4i   
4    24-8i  -5-12i  -1+3i   13-32i   -17-6i   3+4i  -20-30i  -15+11i      4   

       a_4  ...   c_27     a_28    b_28   c_28     a_29     b_29   c_29  \
0   28-23i  ...  -8-6i   29+19i  11-29i  -9+4i    27-9i  -14-22i  -2+8i   
1   26+30i  ...  10-4i  -32+15i  18+25i   2-9i   -9+28i    26+3i  -5-7i   
2   34-17i  ...  -9-4i   33+11i   4-31i  -8+6i   24-14i  -16-19i  -1+8i   
3  -19+32i  ...   8-6i  -24+21i  23+16i  -2-8i    1+29i    24-6i  -7-4i   
4   -35-5i  ...  -2+8i    8-30i  -28-2i   6+7i  -15-21i  -16+18i   8+1i

Under Progress /csi_walk_d2_1
     a_1      b_1    c_1    a_2      b_2    c_2    a_3      b_3    c_3    a_4  \
0  -2+6i   -3-24i  -1+2i   5+7i  -28-18i   1+3i   9+1i  -33+11i   3+1i   8-6i   
1  -4+3i    6-22i  -1+1i   0+6i  -20-23i   0+3i   7+4i   -31+5i   3+1i   8-2i   
2   5+2i   -22+7i   2+2i   6-3i  -12+28i   3-2i   2-8i   16+28i   0-3i  -5-8i   
3      5  -20+13i      2   5-5i   -5+32i   2-2i  -1-8i   24+23i  -1-3i  -7-6i   
4  -4+2i   13-21i  -2+1i  -3+6i  -10-30i  -1+3i   4+7i  -32-11i   2+3i   9+3i   

   ...   c_27   a_28     b_28   c_28    a_29    b_29   c_29   a_30     b_30  \
0  ...  -4+2i   9+7i  -22-21i  -1+4i   10-1i  -25+2i   2+3i   7-4i  -19+10i   
1  ...   3+1i  1-12i    0+28i   3-2i   -7-6i  19+14i  -1-3i  -8-2i    20+3i   
2  ...   4-1i  -5-9i   17+23i   1-3i  -10-3i   24+3i  -2-2i  -8+2i    18-7i   
3  ...   3+3i  6-10i  -13+27i   4-1i   -3-9i   9+22i   2-3i  -5-7i   15+14i   
4  ...  -3+2i   9+8i  -23-18i   0+4i   10-1i  -24+4i   2+2i   7-4i  -16+12i   

    c_30

The shape after time parsing :  (176, 90, 300)
Under Progress /csi_walk_b1e4_1
       a_1     b_1    c_1      a_2      b_2    c_2      a_3     b_3    c_3  \
0   -26-3i  -1+13i   2-2i  -24+27i   14+12i  -1-4i   13+34i   16-7i  -4-1i   
1  -10-26i  -12+5i      3  -34-11i   -5+17i   3-3i  -30+24i  12+16i  -1-4i   
2   -26-7i  -4+13i   2-2i  -29+22i   11+15i   0-4i    6+36i   19-3i  -3-1i   
3    8-25i  -12-4i   2+2i  -20-31i  -15+10i      4   -37-2i  -1+19i   2-4i   
4   -8+26i   13+5i  -2-2i   20+30i   15-10i  -4+1i    38+1i  -1-20i  -2+4i   

       a_4  ...   c_27     a_28     b_28   c_28    a_29     b_29   c_29  \
0   34+11i  ...  -8-2i    33+1i   -5-29i  -6+7i  15-21i  -22-10i   2+7i   
1   -1+38i  ...   8-3i  -28+16i   19+23i   2-9i  -6+27i    25+1i  -5-6i   
2   31+19i  ...  -3-8i   13+29i   25-17i  -9-3i   26+5i    0-26i  -6+5i   
3  -26+27i  ...   5+7i  -19-25i  -19+23i      9  -26-1i    4+24i   5-5i   
4   24-29i  ...  -9-1i       31   -6-29i  -5+6i  17-21i  -22-12i   2+7i   

 

The shape after time parsing :  (176, 90, 300)
Under Progress /csi_walk_a1e5_1
       a_1     b_1    c_1      a_2      b_2     c_2      a_3      b_3     c_3  \
0   18-23i  -8-10i   1+5i  -10-36i   -17+1i    6+2i  -37-11i   -9+17i    5-4i   
1   -8+35i   14+9i  -2-8i   29+33i    20-8i  -10-4i   45-10i    4-24i  -11+7i   
2  -14+33i  11+11i   0-8i   24+39i    20-4i   -7-7i    46+2i    8-21i  -10+4i   
3   24-21i  -6-14i  -3+7i   -6-42i   -19-5i    6+9i  -39-16i  -14+15i      11   
4    32-7i   3-15i  -6+5i   17-38i  -14-15i   0+10i  -24-37i   -21+4i    9+6i   

       a_4  ...     c_27     a_28     b_28     c_28     a_29     b_29    c_29  \
0  -30+23i  ...    12-7i  -32+18i   16+23i    0-12i   -3+31i       25   -8-7i   
1   18-42i  ...  -11+19i   19-33i   -41-4i   10+17i  -15-32i  -25+28i   17+2i   
2   29-34i  ...      -20    39+2i  -14-38i  -11+17i   21-25i  -34-13i   6+17i   
3  -35+22i  ...   14-14i  -25+27i   38+14i   -5-18i    6+29i   28-19i  -14-6i   
4   -41-5i  ...    5+19i   -7

The shape after time parsing :  (176, 90, 300)
Under Progress /csi_walk_d6_1
     a_1      b_1    c_1    a_2      b_2    c_2    a_3      b_3   c_3    a_4  \
0   2+5i   -8-22i  -1+2i   6+2i  -31-12i   2+3i   5-3i  -28+17i     3   3-6i   
1   4+4i  -20-15i   1+2i   6-1i   -31+8i   3+2i   4-6i  -16+32i  3-1i  -1-6i   
2      5      -25   2+2i   4-5i  -20+27i   3-1i  -2-6i   11+34i  2-4i  -5-4i   
3  -4+3i   18-13i  -2-1i  -1+6i    3-30i  -2+2i   4+5i  -22-22i  0+3i   6+1i   
4      5      -23   2+1i   4-5i  -20+23i      3  -1-6i    7+32i  2-3i  -5-4i   

   ...   c_27   a_28     b_28   c_28   a_29     b_29   c_29   a_30     b_30  \
0  ...     -3   6+7i  -16-27i  -2+3i   7+1i   -23-5i   1+3i   6-2i   -20+5i   
1  ...   0+4i   6-7i  -24+18i   4+2i   1-8i   -4+26i   3-1i  -3-6i    7+20i   
2  ...   4-1i  -7-7i   19+24i   2-3i     -8    25+1i  -2-3i  -5+3i    18-9i   
3  ...  -2-3i  -2+9i    9-26i     -3   4+6i  -10-20i  -2+3i   5+3i  -16-12i   
4  ...   3+2i  -1-9i   -2+28i   3-1i  -6-5i   1